In [8]:
import tensorflow as tf 
import pathlib
import numpy as np
from matplotlib import pyplot
from matplotlib import image
import glob

In [9]:
def download_oasis ():
    
    #download oasis brain MRI data
    dataset_url = "https://cloudstor.aarnet.edu.au/plus/s/n5aZ4XX1WBKp6HZ/download"
    data_dir = tf.keras.utils.get_file(origin=dataset_url,fname='oa-sis' ,untar=True)
    data_dir = pathlib.Path(data_dir)
    
    # unzip data to current directory 
    print (data_dir)
    ! unzip /root/.keras/datasets/oa-sis.tar.gz

In [10]:
def load_training (path):
    # load training images (non segmented) in the path and store in numpy array
    image_list = []
    for filename in glob.glob(path+'/*.png'): 
        im=image.imread (filename)
        image_list.append(im)

    print('train_X shape:',np.array(image_list).shape)
    train_set = np.array(image_list, dtype=np.float32)
    return train_set


In [11]:
def process_training (data_set):
    # the method normalizes training images and adds 4th dimention 

    train_set = data_set
    train_set = (train_set - np.mean(train_set))/ np.std(train_set)
    train_set= (train_set- np.amin(train_set))/ np.amax(train_set- np.amin(train_set))
    train_set = train_set [:,:,:,np.newaxis]
    
    return train_set

In [14]:
def load_labels (path):
    # loads labels images and map pixel values to class indices and convert image data type to unit8 

    n_classes = 4
    image_list =[]
    for filename in glob.glob(path+'/*.png'): 
        im=image.imread (filename)
        one_hot = np.zeros((im.shape[0], im.shape[1]))
        for i, unique_value in enumerate(np.unique(im)):
          one_hot[:, :][im == unique_value] = i
        image_list.append(one_hot)

    print('train_y shape:',np.array(image_list).shape)
    labels = np.array(image_list, dtype=np.uint8)
    
    pyplot.imshow(labels[2])
    pyplot.show()

    return labels

In [18]:
def process_labels(seg_data):
    # one hot encode label data and convert to numpy array
    onehot_seg_data = []
    for n in range(seg_data.shape[0]): 
      im = seg_data[n]
      n_classes = 4
      one_hot = np.zeros((im.shape[0], im.shape[1], n_classes),dtype=np.uint8)
      for i, unique_value in enumerate(np.unique(im)):
          one_hot[:, :, i][im == unique_value] = 1
    onehot_seg_data.append(one_hot)
    
    onehot_seg_data =np.array(onehot_seg_data)
    print (onehot_seg_data.dtype)
    #print (np.unique(onehot_validate_Y))
    print (onehot_seg_data.shape)

    return onehot_seg_data


In [19]:
# test script 
#download_oasis()
#train_X = load_training ('/content/keras_png_slices_data/keras_png_slices_train')
#pyplot.imshow(train_X[2])
#pyplot.show()

#train_X = process_training(train_X)
#print (train_X.shape)
#print(np.min (train_X))
#print(np.max (train_X))
#np.unique(train_X[3])
#train_Y = load_labels('/content/keras_png_slices_data/keras_png_slices_seg_train')
train_Y = process_labels(train_Y)

uint8
(1, 256, 256, 4)
